<a href="https://colab.research.google.com/github/Team-c22/Background-remover/blob/main/Background_remover_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [3]:
%cd gdrive/My Drive

/content/gdrive/My Drive


In [4]:
!wget http://download.tensorflow.org/models/deeplabv3_mnv2_pascal_train_aug_2018_01_29.tar.gz

--2023-01-06 15:00:36--  http://download.tensorflow.org/models/deeplabv3_mnv2_pascal_train_aug_2018_01_29.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 209.85.145.128, 2607:f8b0:4001:c1e::80
Connecting to download.tensorflow.org (download.tensorflow.org)|209.85.145.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23882985 (23M) [application/x-tar]
Saving to: ‘deeplabv3_mnv2_pascal_train_aug_2018_01_29.tar.gz’

deeplabv3_mnv2_pasc 100%[===================>]  22.78M  59.6MB/s    in 0.4s    

2023-01-06 15:00:36 (59.6 MB/s) - ‘deeplabv3_mnv2_pascal_train_aug_2018_01_29.tar.gz’ saved [23882985/23882985]



In [5]:
!wget http://download.tensorflow.org/models/deeplabv3_pascal_train_aug_2018_01_04.tar.gz

--2023-01-06 15:02:29--  http://download.tensorflow.org/models/deeplabv3_pascal_train_aug_2018_01_04.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 209.85.145.128, 2607:f8b0:4001:c1e::80
Connecting to download.tensorflow.org (download.tensorflow.org)|209.85.145.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 460058541 (439M) [application/x-tar]
Saving to: ‘deeplabv3_pascal_train_aug_2018_01_04.tar.gz’

deeplabv3_pascal_tr 100%[===================>] 438.75M  65.0MB/s    in 6.6s    

2023-01-06 15:02:36 (66.7 MB/s) - ‘deeplabv3_pascal_train_aug_2018_01_04.tar.gz’ saved [460058541/460058541]



In [6]:
!mkdir mobile_net_model

mkdir: cannot create directory ‘mobile_net_model’: File exists


In [7]:
!mkdir xception_model+

In [8]:
!tar xvzf deeplabv3_mnv2_pascal_train_aug_2018_01_29.tar.gz -C mobile_net_model --strip=1

deeplabv3_mnv2_pascal_train_aug/model.ckpt-30000.index
deeplabv3_mnv2_pascal_train_aug/frozen_inference_graph.pb
deeplabv3_mnv2_pascal_train_aug/model.ckpt-30000.data-00000-of-00001


In [9]:
!tar xvzf deeplabv3_pascal_train_aug_2018_01_04.tar.gz -C xception_model --strip=1

deeplabv3_pascal_train_aug/model.ckpt.index
deeplabv3_pascal_train_aug/frozen_inference_graph.pb
deeplabv3_pascal_train_aug/model.ckpt.data-00000-of-00001


In [10]:
!rm deeplabv3_mnv2_pascal_train_aug_2018_01_29.tar.gz
!rm deeplabv3_pascal_train_aug_2018_01_04.tar.gz

In [11]:
import os
from io import BytesIO

import numpy as np
from PIL import Image

import tensorflow as tf
import sys
import datetime
import matplotlib.pyplot as plt

class DeepLabModel(object):
  """Class to load deeplab model and run inference."""

  INPUT_TENSOR_NAME = 'ImageTensor:0'
  OUTPUT_TENSOR_NAME = 'SemanticPredictions:0'
  INPUT_SIZE = 513
  FROZEN_GRAPH_NAME = 'frozen_inference_graph'

  def __init__(self, tarball_path):
    """Creates and loads pretrained deeplab model."""
    self.graph = tf.Graph()

    graph_def = None
    graph_def = tf.compat.v1.GraphDef.FromString(open(tarball_path + "/frozen_inference_graph.pb", "rb").read()) 


    if graph_def is None:
      raise RuntimeError('Cannot find inference graph in tar archive.')

    with self.graph.as_default():
      tf.import_graph_def(graph_def, name='')

    self.sess = tf.compat.v1.Session(graph=self.graph)

  def run(self, image):
    """Runs inference on a single image.
    Args:
      image: A PIL.Image object, raw input image.
    Returns:
      resized_image: RGB image resized from original input image.
      seg_map: Segmentation map of `resized_image`.
    """
    start = datetime.datetime.now()

    width, height = image.size
    resize_ratio = 1.0 * self.INPUT_SIZE / max(width, height)
    target_size = (int(resize_ratio * width), int(resize_ratio * height))
    resized_image = image.convert('RGB').resize(target_size, Image.ANTIALIAS)
    batch_seg_map = self.sess.run(
        self.OUTPUT_TENSOR_NAME,
        feed_dict={self.INPUT_TENSOR_NAME: [np.asarray(resized_image)]})
    seg_map = batch_seg_map[0]

    end = datetime.datetime.now()

    diff = end - start
    print("Time taken to evaluate segmentation is : " + str(diff))

    return resized_image, seg_map

def drawSegment(baseImg, matImg):
  width, height = baseImg.size
  dummyImg = np.zeros([height, width, 4], dtype=np.uint8)
  for x in range(width):
            for y in range(height):
                color = matImg[y,x]
                (r,g,b) = baseImg.getpixel((x,y))
                if color == 0:
                    dummyImg[y,x,3] = 0
                else :
                    dummyImg[y,x] = [r,g,b,255]
  img = Image.fromarray(dummyImg)
  plt.figure(figsize=(12, 10))
  plt.subplot(221)
  plt.imshow(baseImg)
  plt.subplot(222)
  plt.imshow(img)


inputFilePath = sys.argv[1]
outputFilePath = sys.argv[2]

if inputFilePath is None or outputFilePath is None:
  print("Bad parameters. Please specify input file path and output file path")
  exit()

modelType = "mobile_net_model"
if len(sys.argv) > 3 and sys.argv[3] == "1":
  modelType = "xception_model"

MODEL = DeepLabModel(modelType)
print('model loaded successfully : ' + modelType)

def run_visualization(filepath):
  """Inferences DeepLab model and visualizes result."""
  try:
  	print("Trying to open : " + sys.argv[1])
  	# f = open(sys.argv[1])
  	jpeg_str = open(filepath, "rb").read()
  	orignal_im = Image.open(BytesIO(jpeg_str))
  except IOError:
    print('Cannot retrieve image. Please check file: ' + filepath)
    return

  print('running deeplab on image %s...' % filepath)
  resized_im, seg_map = MODEL.run(orignal_im)

  # vis_segmentation(resized_im, seg_map)
  drawSegment(resized_im, seg_map)

model loaded successfully : mobile_net_model


In [12]:
run_visualization('https://pbs.twimg.com/profile_images/691700243809718272/z7XZUARB_400x400.jpg')

Trying to open : -f
Cannot retrieve image. Please check file: https://pbs.twimg.com/profile_images/691700243809718272/z7XZUARB_400x400.jpg
